источник данных - Вакансии всех регионов России из ЕЦП «Работа в России» (https://trudvsem.ru/opendata/api)

### На какие вопросы можно ответить?

1. какое распределение вакансий по регионам? Абсолютные цифры и на душу населения
2. в каких регионах требуется больше специалистов с ученой степенью?
3. какое распределение вакансий по категориям и регионам?
4. средние зарплаты?
5. динамика зарплат по времени?
6. в каких категориях не указывают зарплаты?

и т.д.100500 вопросов можно придумать)

Также можно будет посмотреть исторические данные и попытаться ответить на вопрос, повлияло ли как-то то или иное событие на рынок вакансий.


### Что предлагается сделать?

Написать приложение, которое будет:
1. скачивать периодически данные из https://trudvsem.ru/opendata/api и сохранять в БД
2. визуализировать данные в BI системе (различные срезы, сегменты и по времени)

Инструменты:
1. clickhouse
2. airflow
3. grafana/datalens
4. python/jupyter notebook
5. docker

### Загрузка данных

1. Данные грузятся кусками (параметр limit + offset)
2. Данные за период можно загрузить, передав в url параметры modifiedFrom и modifiedTo

Загрузим для примера записи за 2024-02-17

In [1]:
import requests
import pandas as pd


url = 'http://opendata.trudvsem.ru/api/v1/vacancies'
date_start = '2024-02-17'
date_end = '2024-02-17'


def load_data(url: str = None,
              offset: int = 1,
              limit: int = 500,
              date_start: str = None,
              date_end: str = None):
    url = f'{url}?offset={offset}&limit={limit}&modifiedFrom={date_start}T00:00:00Z&modifiedTo={date_end}T23:59:00Z'
    data = requests.get(url)
    data = [vac['vacancy'] for vac in data.json()['results']['vacancies']]
    data = pd.DataFrame.from_dict(data, orient='columns')
    # adding company attributes
    data['company_name'] = data['company'].apply(lambda x: x['name'], 1)
    data['companycode'] = data['company'].apply(lambda x: x['companycode'], 1)
    data['hr-agency'] = data['company'].apply(lambda x: x['hr-agency'], 1)
    # adding region name
    data['region_name'] = data['region'].apply(lambda x: x['name'], 1)
    # adding specialization
    data['specialisation'] = data['category'].apply(lambda x: x['specialisation'], 1)
    return data

In [2]:
offset = 1
df = pd.DataFrame()
while True:
    offset = offset+1
    limit = 100
    try:
        data = load_data(url=url,
                         offset=offset,
                         limit=limit,
                         date_start=date_start,
                         date_end=date_end)
        df = pd.concat([df, data])
    except KeyError as e:
        break

In [3]:
df['specialisation'].value_counts()

specialisation
Производство                                                        237
Продажи, закупки, снабжение, торговля                               196
Образование, наука                                                  148
Транспорт, автобизнес, логистика, склад, ВЭД                        144
Здравоохранение и социальное обеспечение                            142
Рабочие специальности                                               117
Машиностроение                                                      111
ЖКХ, эксплуатация                                                    70
Строительство, ремонт, стройматериалы, недвижимость                  65
Безопасность, службы охраны                                          62
Государственная служба, некоммерческие организации                   60
Электроэнергетика                                                    55
Информационные технологии, телекоммуникации, связь                   54
Работы, не требующие квалификации                

In [4]:
df['employment'].value_counts()

employment
Полная занятость       1744
Частичная занятость      51
Удаленная                49
Временная                47
Стажировка                3
Сезонная                  2
Name: count, dtype: int64

In [5]:
df['company_name'].nunique()

470

In [6]:
# средняя зарплата (сначала почистим поле от букв и преобразуем к int)
df['salary'] = df['salary'].fillna("0").apply(lambda x: int(x.replace("от ", "")), 1)
df[df['salary']>0].groupby('employment').agg({'salary': ['mean', 'count', 'max']})

salary              
                             mean count     max
employment                                     
Временная            32022.282609    46   81464
Полная занятость     48291.824693  1466  315000
Сезонная             25000.000000     2   25000
Стажировка           28821.000000     2   38400
Удаленная            32416.666667    48   45000
Частичная занятость  32008.586957    46  100000

In [7]:
df[df['salary']>0].groupby('region_name').agg({'salary': ['mean', 'count', 'max']})

salary              
                                          mean count     max
region_name                                                 
Алтайский край                    28651.500000     8   40000
Амурская область                  25987.000000     1   25987
Архангельская область             52759.461538    26   81000
Астраханская область              42934.454545    11   80000
Белгородская область              34040.333333     6   58000
...                                        ...   ...     ...
Челябинская область               63400.000000     6  200000
Чувашская Республика - Чувашия    25333.333333     3   30000
Чукотский автономный округ       120000.000000     2  120000
Ямало-Ненецкий автономный округ  120000.000000     1  120000
Ярославская область               53547.619048    21   80000

[76 rows x 3 columns]

In [8]:
df[df['salary']>0].groupby('specialisation').agg({'salary': ['mean', 'count', 'max']})

salary              
                                                            mean count     max
specialisation                                                                
Административная работа, секретариат, АХО           33172.482759    29   80000
Банки, кредит, страхование, пенсионное обеспечение  46936.363636    22  115000
Безопасность, службы охраны                         34756.117647    51  115000
Бухгалтерия, налоги, управленческий учет            35270.341463    41   80000
Высший менеджмент                                   25000.000000     1   25000
Государственная служба, некоммерческие организации  87895.000000    60  315000
Добывающая промышленность                           68500.000000     6  200000
Домашний персонал                                   35000.000000     3   35000
ЖКХ, эксплуатация                                   47409.900000    70  140000
Здравоохранение и социальное обеспечение            53173.348148   135  150000
Информационные технологии, телекоммуникации, связь  42269.526316    38  130000
Искусство, культура и развлечения                   34325.800000    15   80000
Кадровая служба, управление персоналом              35706.344828    29   72000
Консалтинг, стратегическое развитие, управление     31954.333333     3   40000
Легкая промышленность                               47441.400000    30   80000
Лесная, деревообрабатывающая, целлюлозно-бумажн...  47998.400000    10  100000
Маркетинг, реклама, PR                              34025.000000     4   39100
Машиностроение                                      51827.504505   111  180000
Металлургия, металлообработка                       40076.923077    13   50000
Образование, наука                                  28516.979310   145  150000
Пищевая промышленность                              34626.468750    32   70000
Продажи, закупки, снабжение, торговля               46465.031496   127  150000
Производство                                        51444.902326   215  177125
Работы, не требующие квалификации                   31534.321429    28  100000
Рабочие специальности                               44627.213333    75  130000
Сельское хозяйство, экология, ветеринария           25056.705882    17   40000
Спорт, фитнес, салоны красоты                       33906.588235    17   80000
Строительство, ремонт, стройматериалы, недвижим...  50790.761905    63  155000
Транспорт, автобизнес, логистика, склад, ВЭД        49199.559140    93  150000
Туризм, гостиницы, рестораны                        40245.555556    36   78000
Услуги населению, сервисное обслуживание            42497.800000    20  110000
Химическая, нефтехимическая, топливная промышле...  52833.333333     6   75000
Электроэнергетика                                   73839.345455    55  165000
Юриспруденция                                       42474.200000    10   60000

In [9]:
df[df['salary']>0]['salary'].describe()

count      1610.000000
mean      46835.327329
std       32025.576126
min        5000.000000
25%       28000.000000
50%       40000.000000
75%       56750.000000
max      315000.000000
Name: salary, dtype: float64